In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
words = open('makemore/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [6]:
block_size = 3

def build_dataset(word):
    X, y = [], []
    
    for w in words:
        context = [0] * block_size
        
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            y.append(ix)
            context = context[1:] + [ix]
            
    X = torch.tensor(X)
    y = torch.tensor(y)
    print(X.shape, y.shape)
    return X, y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, ytr = build_dataset(words[:n1])
Xdev, ydev = build_dataset(words[n1:n2])
Xte, yte = build_dataset(words[n2:])

torch.Size([228146, 3]) torch.Size([228146])
torch.Size([228146, 3]) torch.Size([228146])
torch.Size([228146, 3]) torch.Size([228146])


In [7]:
# MLP revised
n_embed = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embed), generator=g)
w1 = torch.randn((n_embed*block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
w2 = torch.randn((n_hidden, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)

parameters = [C, w1, b1, w2, b2]

print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

11897


In [9]:
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, yb = Xtr[ix], ytr[ix]
    
    #forward
    emb = C[Xb] #embed
    embcat = emb.view(emb.shape[0], -1) #concatenate vectors
    hpreact = embcat @ w1 + b1 #hidden layer pre activation
    h = torch.tanh(hpreact)
    logits = h @ w2 + b2
    loss = F.cross_entropy(logits, yb)
    
    #backward
    for p in parameters:
        p.grad = None
    loss.backward()
    
    #update
    lr = 0.1 if i < 100000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
    
    if i % 10000 == 0:
        print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    

      0/ 200000: 26.6230
  10000/ 200000: 2.5342
  20000/ 200000: 2.1831
  30000/ 200000: 2.2873
  40000/ 200000: 2.1919
  50000/ 200000: 2.3506
  60000/ 200000: 2.3413
  70000/ 200000: 2.2091
  80000/ 200000: 1.9438
  90000/ 200000: 2.1607
 100000/ 200000: 2.2768
 110000/ 200000: 2.2037
 120000/ 200000: 2.2527
 130000/ 200000: 1.9775
 140000/ 200000: 1.7269
 150000/ 200000: 2.4453
 160000/ 200000: 2.1155
 170000/ 200000: 2.2949
 180000/ 200000: 2.1016
 190000/ 200000: 2.0198


In [13]:
@torch.no_grad()
def split_loss(split):
    x,y = {
        'train': (Xtr, ytr),
        'val': (Xdev, ydev),
        'test': (Xte, yte)
    }[split]
    
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    h = torch.tanh(embcat @ w1 + b1)
    logits = h @ w2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
split_loss('train')
split_loss('val')

train 2.1253464221954346
val 2.1253464221954346


In [15]:
# sampe from the model 
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    out = []
    context = [0] * block_size
    
    while True: 
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(1, -1) @ w1 + b1)
        logits = h @ w2 + b2
        probs = F.softmax(logits, dim=1)
        
        ix = torch.multinomial(probs, num_samples=1, generator=g).item()
        
        context = context[1:] + [ix]
        out.append(ix)
        
        if ix == 0:
            break
            
    print(''.join(itos[i] for i in out))

carmahzamillivi.
kimrix.
taty.
skanden.
jazhuel.
delynn.
jareen.
nellara.
chaiir.
kaleigh.
ham.
prin.
quint.
sulin.
alianni.
wraelo.
dearynn.
jace.
pirraylen.
edi.


In [18]:
# we should know what to expect 

-torch.tensor(1/27.0).log() # this is the loss we would expect at initialization

tensor(3.2958)